In [1]:
import cv2
from func_face import face_detect_dl
import face_recognition
import pickle
from imutils import paths
from PIL import Image
import datetime
import os
import pandas as pd
import time
import shutil

In [2]:
data = pickle.loads(open(r'C:\Users\user\Documents\09.Python\Face Recognition (My)\encodings.pickle', "rb").read())

In [3]:
from_p = r'C:\Users\user\Documents\08. Data_IN\Camera'
csv_file = r'C:\Users\user\Documents\08. Data_IN\face_recog.csv'
to_p   = r'C:\Users\user\Documents\08. Data_IN\DB_Rocognition'
imagePaths = list(paths.list_images(from_p))

In [4]:
columns=['DateTime','Name', 'Camera', 'Path']
df = pd.DataFrame(columns = columns)

# Functions

In [5]:
def convert_xy(boxes):
    ls=[]
    for b in boxes:
        (startY, startX,  endY,  endX) = b
        ls.append((startY, endX, endY, startX))
        
    return ls 

In [6]:
def check_xy(img, xy):
    startY, endX, endY, startX = xy
    h = img.shape[0]
    w = img.shape[1]
    if (startY>h) | (endY>h) | (startX>w) | (endX>w):
        return True
    else:
        return False  

In [7]:
def recgnize(image, boxes, tol=0.45 ):
    #rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    encodings = face_recognition.face_encodings(image, boxes)
    names = []
    for encoding in encodings:
        matches = face_recognition.compare_faces(data["encodings"],
                                                 encoding,tolerance=tol)
        name = "Unknown"
        
        if True in matches:
            #print(matches,'\n')
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            #print(matchedIdxs)
            counts = {}
            for i in matchedIdxs:
                name = data["names"][i]
                #print(name)
                counts[name] = counts.get(name, 0)+1
            name = max(counts, key=counts.get)
            #print(counts)
        names.append(name)
    
    return names

In [8]:
def save_pic(jmage, patch, imagePath):
    
    name_f = imagePath.split(os.path.sep)[-1]
    
    delim='_'
    pic = Image.fromarray(jmage)
    dt = datetime.datetime.now()
    name_file = '\\' + name_f #+ str(dt.year)+delim+str(dt.month)+delim+str(dt.day)+delim+str(dt.hour)+delim+str(dt.minute)+delim+str(dt.second)+delim+str(dt.microsecond)+'.jpg'
    
    try:
        pic.save(patch+name_file, "JPEG")  
    except:
        os.mkdir(patch)
        pic.save(patch+name_file, "JPEG")
        
    return patch+name_file

In [9]:
def prepere_pack(imagePaths):
    lab_save = []
    for imagePath in imagePaths:
        img_unk = cv2.imread(imagePath)
        rgb_unk = cv2.cvtColor(img_unk, cv2.COLOR_BGR2RGB)
        
        box_my = face_detect_dl(rgb_unk)
        box = convert_xy(box_my) 
        names = recgnize(rgb_unk,box,0.5)
        
        for ((startY, endX, endY, startX), name) in zip(box, names):
            
            patch_to = to_p + '\\' + name 
            if (name == 'Unknown') & (imagePath == imagePaths[len(imagePaths)-1]):
                img_unkown = cv2.imread(imagePaths[len(imagePaths)//2])
                rgb_unkown = cv2.cvtColor(img_unkown, cv2.COLOR_BGR2RGB)
                p = save_pic(rgb_unkown, patch_to, imagePaths[len(imagePaths)//2])
                continue
            elif name == 'Unknown':
                continue
                
            cv2.rectangle(rgb_unk, (startX, startY), (endX, endY),(0, 0, 255), 2)
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(rgb_unk, name, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
            
            if lab_save.count(name)==0:
                p = save_pic(rgb_unk, patch_to, imagePath) 
                lab_save.append(name)    
    

In [10]:
from_p = r'C:\Users\user\Documents\08. Data_IN\Camera\test'
#imagePaths = list(paths.list_images(from_p))

In [12]:
t0 = time.time()
i=0
path_pkg = from_p+ '\\pack'
while os.path.exists(path_pkg+str(i)):
    PkgPaths = list(paths.list_images(path_pkg+str(i)))
    prepere_pack(PkgPaths)
    #print(path_pkg+str(i),' - готов')
    shutil.rmtree(path_pkg+str(i))
    i+=1 
    
print (time.time()-t0)

31.46770429611206


In [11]:
with open(csv_file, 'a+') as file:
    df.to_csv(file,index =False, header = None)

In [219]:
(r'C:\Users\user\Documents\08. Data_IN\Camera\test\pack54')

# Next code was moved to function

In [166]:
t0 = time.time()
lab_save = []
for imagePath in imagePaths:
    img_unk = cv2.imread(imagePath)
    rgb_unk = cv2.cvtColor(img_unk, cv2.COLOR_BGR2RGB)
        
    box_my = face_detect_dl(rgb_unk)
    box = convert_xy(box_my)   
    #rgb_unk = cv2.fastNlMeansDenoisingColored(rgb_unk,None,3,3,7,21)
    names = recgnize(rgb_unk,box,0.5)
    #print(names)

    
    for ((startY, endX, endY, startX), name) in zip(box, names):
        
        #if check_xy(rgb_unk, (startY, endX, endY, startX)):
        #    continue  
          
        patch_to = to_p + '\\' + name 
        
        if (name == 'Unknown') & (imagePath == imagePaths[len(imagePaths)-1]):
            img_unkown = cv2.imread(imagePaths[len(imagePaths)//2])
            rgb_unkown = cv2.cvtColor(img_unkown, cv2.COLOR_BGR2RGB)
            p = save_pic(rgb_unkown, patch_to, imagePaths[len(imagePaths)//2])
            continue
        elif name == 'Unknown':
            continue
            
        
        cv2.rectangle(rgb_unk, (startX, startY), (endX, endY),(0, 0, 255), 2)
        y = startY - 15 if startY - 15 > 15 else startY + 15
        cv2.putText(rgb_unk, name, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
        #cv2.putText(rgb_unk, str(endY - startY) + ' X ' + str(endX-startX), (endX, endY), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
        if lab_save.count(name)==0:
            p = save_pic(rgb_unk, patch_to, imagePath)  
            lab_save.append(name)
        #elif names.count('Unknown') > 0:
        #    p = save_pic(rgb_unk, patch_to, imagePath)
            
        
        
        #запись в DataFrame
        
        name_camera = imagePath.split(os.path.sep)[-2]
        name_file = imagePath.split(os.path.sep)[-1]
        dt = name_file.split('_')[:6]
        dt = list(map(int, dt))
        dt = pd.datetime(dt[0],dt[1],dt[2],dt[3],dt[4],dt[5])
        
        df.loc[len(df)] = {'DateTime':str(dt),
                           'Name':name,
                           'Camera':name_camera,
                           'Path':p            
                          }
    #os.remove(imagePath)
        
df.sort_values('DateTime', inplace = True)   

print (time.time()-t0)

['Unknown']